# Binding prediction using AptaNet
Step-by-step guide for using AptaNet for binary aptamer–protein binding prediction.

## Overview
This notebook showcases the AptaNet algorithm, a deep learning method that combines sequence-derived (k-mer + PSeAAC) features with RandomForest-based feature selection, and a multi-layer perceptron to predict whether an aptamer and a protein interact (binary classification). An overview of the classes and helper functions used in this notebook:

- **pairs_to_features**: helper that converts `(aptamer_sequence, protein_sequence)` pairs into feature vectors using k-mer + PSeAAC.
- **FeatureSelector**: a Random Forest-based transformer that selects important features.
- **SkorchAptaNet**: a PyTorch MLP wrapped in Skorch for binary classification.
- **load_pfoa_structure**: helper to load the PFOA molecule structure from PDB file.

## Data preparation
To train the `AptaNetMLP` the notebook uses:
* For `X`:
    * 5 random aptamer sequences of length>30 (to satisfy the default lambda value of 30 set in the PSeAAC algorithm).
    * Amino acid sequences extracted from the 1GNH protein molecule.
    
    The aptamer sequences and the amino acid sequences form tuples `(aptamer_sequence, protein_sequence)` to form `X`.
* For `y`:
    * A random binary value (to indicate if the aptamer binds or not) as dummy data.

In [1]:
# Data imports
import torch

from pyaptamer.datasets import load_1gnh_structure
from pyaptamer.utils.struct_to_aaseq import struct_to_aaseq

In [2]:
aptamer_sequence = [
    "GGGAGGACGAAGACGACUCGAGACAGGCUAGGGAGGGA",
    "AAGCGUCGGAUCUACACGUGCGAUAGCUCAGUACGCGGU",
    "CGGUAUCGAGUACAGGAGUCCGACGGAUAGUCCGGAGC",
    "UAGCUAGCGAACUAGGCGUAGCUUCGAGUAGCUACGGAA",
    "GCUAGGACGAUCGCACGUGACCGUCAGUAGCGUAGGAGA",
]

gnh = load_1gnh_structure()
protein_sequence = struct_to_aaseq(gnh)

# Build all combinations (aptamer, protein)
X = [(a, p) for a in aptamer_sequence for p in protein_sequence]

# Dummy binary labels for the pairs
y = torch.randint(0, 2, (len(X),), dtype=torch.float32)

## Build your own pipeline
### The Neighbourhood cleaning rule
In the `AptaNet` paper, the authors mention using the `NeighbourhoodCleaningRule` from `imblearn` in order to balance the dataset, as in their dataset they had more negative (0) values than positives (1).

 To build a scikit-learn pipeline, follow these steps:
1. Convert the input to the desired (aptamer_sequence, protein_sequence) format.
    * OPTIONAL: As mentioned in the paper, perform under-sampling using the  
    Neighborhood Cleaning Rule to balance the classes.
2. Get the PSeAAC feature vectors for your converted input (using `pairs_to_features`).
3. Select the number of features to use from the feature vector (using `FeatureSelector`).
4. Define the skorch neural network (using `AptaNetMLP`).
### Different workflows
In this notebook we will cover 3 different workflows one can follow, in ascending order of customizability:

1. A minimal workflow with no dataset balancing, while using the in-built pipeline.
2. Using your own custom pipeline.
3. Dataset balancing, while using your own pipeline.

### First workflow
A minimal workflow with no dataset balancing, while using the in-built pipeline.

In [3]:
# If you want to use the AptaNet pipeline, you can import it directly
from pyaptamer.aptanet import AptaNetPipeline

In [4]:
pipeline = AptaNetPipeline()

In [5]:
# Fit the pipeline on the aptamer-protein pairs
pipeline.fit(X, y)

# Predict the labels for the training data
y_pred = pipeline.predict(X)

In [6]:
# Optional: Evaluate training accuracy
from sklearn.metrics import accuracy_score

print("Training Accuracy:", accuracy_score(y, y_pred))

Training Accuracy: 0.54


### Second Worflow

Your own custom-built pipeline.

In [7]:
# If you to build your own aptamer pipeline, you should use the following imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from skorch import NeuralNetBinaryClassifier

from pyaptamer.aptanet._aptanet_nn import AptaNetMLP
from pyaptamer.utils._aptanet_utils import pairs_to_features

In [8]:
feature_transformer = FunctionTransformer(
    func=pairs_to_features,
    validate=False,
    # Optional arguments for pairs_to_features
    # example: kw_args={'k': 4, 'pseaac_kwargs': {'lambda_value': 30}}
    kw_args={},
)

selector = SelectFromModel(
    estimator=RandomForestClassifier(
        n_estimators=300,
        max_depth=9,
        random_state=None,
    ),
    threshold="mean",
)

# Define the classifier
net = NeuralNetBinaryClassifier(
    module=AptaNetMLP,
    module__input_dim=None,
    module__hidden_dim=128,
    module__n_hidden=7,
    module__dropout=0.3,
    module__output_dim=1,
    module__use_lazy=True,
    criterion=torch.nn.BCEWithLogitsLoss,
    max_epochs=200,
    lr=0.00014,
    optimizer=torch.optim.RMSprop,
    optimizer__alpha=0.9,
    optimizer__eps=1e-08,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

pipeline = Pipeline(
    [
        ("features", feature_transformer),
        ("selector", selector),
        ("clf", net),
    ]
)

In [9]:
# Fit the pipeline on the aptamer-protein pairs
pipeline.fit(X, y)

# Predict the labels for the training data
y_pred = pipeline.predict(X)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7086       0.4000        0.7011  0.0080
      2        0.7620       0.4000        0.7009  0.0105
      3        0.7134       0.4000        0.7007  0.0110
      4        0.7606       0.4000        0.7006  0.0080
      5        0.7394       0.4000        0.7005  0.0090
      6        0.7378       0.4000        0.7004  0.0100
      7        0.6926       0.4000        0.7003  0.0090
      8        0.7217       0.4000        0.7002  0.0090
      9        0.7119       0.4000        0.7001  0.0100
     10        0.7540       0.4000        0.7000  0.0090
     11        0.7884       0.4000        0.6999  0.0085
     12        0.6804       0.4000        0.6998  0.0090
     13        0.7759       0.4000        0.6997  0.0090
     14        0.7175       0.4000        0.6996  0.0080
     15        0.8031       0.4000        0.6996  0.0090
     16        0.7258       0.4

In [10]:
# Optional: Evaluate training accuracy
from sklearn.metrics import accuracy_score

print("Training Accuracy:", accuracy_score(y, y_pred))

Training Accuracy: 0.54


### Third workflow
Dataset balancing using under-sampling, while using your own pipeline.

In [11]:
# If you to build your own aptamer pipeline, you should use the following imports
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import FunctionTransformer
from skorch import NeuralNetBinaryClassifier

from pyaptamer.aptanet._aptanet_nn import AptaNetMLP
from pyaptamer.utils._aptanet_utils import pairs_to_features

In [12]:
# OPTIONAL: If you want to use the Neighborhood Cleaning Rule for under-sampling
# NOTE: If you want to use the under-sampling, you need to install imbalanced-learn
# and use the Pipeline from imblearn
# %pip install imblearn
from imblearn.pipeline import Pipeline
from imblearn.under_sampling import NeighbourhoodCleaningRule  # noqa: E402

In [13]:
feature_transformer = FunctionTransformer(
    func=pairs_to_features,
    validate=False,
    # Optional arguments for pairs_to_features
    # example: kw_args={'k': 4, 'pseaac_kwargs': {'lambda_value': 30}}
    kw_args={},
)

selector = SelectFromModel(
    estimator=RandomForestClassifier(
        n_estimators=300,
        max_depth=9,
        random_state=None,
    ),
    threshold="mean",
)

# Define the classifier
net = NeuralNetBinaryClassifier(
    module=AptaNetMLP,
    module__input_dim=None,
    module__hidden_dim=128,
    module__n_hidden=7,
    module__dropout=0.3,
    module__output_dim=1,
    module__use_lazy=True,
    criterion=torch.nn.BCEWithLogitsLoss,
    max_epochs=200,
    lr=0.00014,
    optimizer=torch.optim.RMSprop,
    optimizer__alpha=0.9,
    optimizer__eps=1e-08,
    device="cuda" if torch.cuda.is_available() else "cpu",
)

pipeline = Pipeline(
    [
        ("features", feature_transformer),
        # Optional under-sampling, use sklearn's Pipeline if you do not need it
        ("ncr", NeighbourhoodCleaningRule()),
        ("selector", selector),
        ("clf", net),
    ]
)

In [14]:
# Fit the pipeline on the aptamer-protein pairs
pipeline.fit(X, y)

# Predict the labels for the training data
y_pred = pipeline.predict(X)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7023       0.3750        0.7047  0.0115
      2        0.6523       0.3750        0.7045  0.0110
      3        0.7088       0.3750        0.7042  0.0110
      4        0.7553       0.3750        0.7041  0.0090
      5        0.7432       0.3750        0.7039  0.0090
      6        0.7215       0.3750        0.7037  0.0080
      7        0.7044       0.3750        0.7036  0.0083
      8        0.7064       0.3750        0.7034  0.0110
      9        0.6890       0.3750        0.7032  0.0090
     10        0.7676       0.3750        0.7030  0.0090
     11        0.7384       0.3750        0.7028  0.0100
     12        0.6737       0.3750        0.7027  0.0080
     13        0.7397       0.3750        0.7025  0.0080
     14        0.7553       0.3750        0.7024  0.0100
     15        0.6919       0.3750        0.7023  0.0090
     16        0.7337       0.3

In [15]:
# Optional: Evaluate training accuracy
from sklearn.metrics import accuracy_score

print("Training Accuracy:", accuracy_score(y, y_pred))

Training Accuracy: 0.46
